<a href="https://colab.research.google.com/github/JuelGeorge/Adaptive_BCI_2D/blob/main/CNNerrp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
while 1:
  pass

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io

In [ ]:
events = scipy.io.loadmat('/content/drive/MyDrive/data/all_events512.mat')['all_events']
data = scipy.io.loadmat('/content/drive/MyDrive/data/all_data512.mat')['all_data']
events = events.T
cz_fz=data[46:48,:,:]
data=cz_fz.reshape((6437,2,512))

In [ ]:
events[events == 1] = 0   #ERRP
events[events == 2] = 1   #NOERRP

In [ ]:
np.shape(events)

(6437, 1)

In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers import *

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

Changes
1. Flatten instead of global average
2. Doubled the filter depth in some layers to increase learnables

In [25]:
model = Sequential()
reg = tf.keras.regularizers.L2(l2=0.0001)
model.add(InputLayer(input_shape=(1, 512,1)))

#Stage 1
model.add(Conv2D(filters=32, kernel_size=(1,64), kernel_regularizer=reg,padding='valid'))

#Stage 2
model.add(Conv2D(filters=32, kernel_size=(1,64), activation='relu', kernel_regularizer=reg,padding='valid'))
model.add(MaxPool2D(pool_size=(1,2)))

#Stage 3
model.add(Conv2D(filters=64, kernel_size=(1,32), activation='relu', kernel_regularizer=reg,padding='valid'))
model.add(MaxPool2D(pool_size=(1,2)))

#Stage 4
model.add(Conv2D(filters=64, kernel_size=(1,16), activation='relu', kernel_regularizer=reg,padding='valid'))
model.add(MaxPool2D(pool_size=(1,2)))

#Stage 5
model.add(Flatten())
model.add(Dense(units=2, activation='softmax', kernel_regularizer=reg))
#model.add(GlobalAveragePooling2D())

In [27]:
opt = tf.keras.optimizers.experimental.SGD(learning_rate=0.001, momentum=0.9)
model.compile(opt, loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [26]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_40 (Conv2D)          (None, 1, 449, 32)        2080      
                                                                 
 conv2d_41 (Conv2D)          (None, 1, 386, 32)        65568     
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 1, 193, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_42 (Conv2D)          (None, 1, 162, 64)        65600     
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 1, 81, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_43 (Conv2D)          (None, 1, 66, 64)       

In [28]:
from sklearn.model_selection import train_test_split
dataTrain, dataTest, eventsTrain, eventsTest = train_test_split(data, events, test_size = 0.2)

NameError: ignored

In [ ]:
np.shape(eventsTrain)

(5149, 1)

In [ ]:
model.fit(dataTrain, eventsTrain, batch_size = 512, epochs = 500,validation_data=(dataTest,eventsTest))

Epoch 1/100
81/81 [==============================] - 13s 142ms/step - loss: 0.7028 - accuracy: 0.3795
Epoch 2/100
81/81 [==============================] - 12s 144ms/step - loss: 0.7027 - accuracy: 0.3863
Epoch 3/100
81/81 [==============================] - 11s 141ms/step - loss: 0.7027 - accuracy: 0.3913
Epoch 4/100
81/81 [==============================] - 11s 140ms/step - loss: 0.7027 - accuracy: 0.4115
Epoch 5/100
81/81 [==============================] - 11s 141ms/step - loss: 0.7027 - accuracy: 0.4129
Epoch 6/100
81/81 [==============================] - 11s 139ms/step - loss: 0.7027 - accuracy: 0.4135
Epoch 7/100
81/81 [==============================] - 11s 139ms/step - loss: 0.7027 - accuracy: 0.4195
Epoch 8/100
81/81 [==============================] - 11s 140ms/step - loss: 0.7027 - accuracy: 0.4267
Epoch 9/100
81/81 [==============================] - 11s 141ms/step - loss: 0.7027 - accuracy: 0.4337
Epoch 10/100
81/81 [==============================] - 11s 141ms/step - loss: 0.702

In [ ]:
eventsPred = model.predict(dataTest)
eventsClasses = np.argmax(eventsPred,axis=1)

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
n=23
print(eventsClasses[n])
print(eventsTest[n])